In this notebook we will test our models for 11 musical instruments on the test dataset, and save the results in the .json file "LUMENDS2023_Final_predictions.json"

In [6]:
%matplotlib inline

from IPython.display import Audio
from IPython.display import Image


import sklearn.preprocessing
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np 
import librosa as lr
import librosa.display
import matplotlib.pyplot as plt
import wave
import soundfile as sf
import math
import visualkeras
import json

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [19]:
username = "user"
directory = "C:/Users/" + username + "/Downloads/drive-download-20230507T145001Z-001/"
file_names = os.listdir(directory)
print(file_names)
print(len(file_names))

['track_0.wav', 'track_10.wav', 'track_100.wav', 'track_101.wav', 'track_104.wav', 'track_105.wav', 'track_106.wav', 'track_107.wav', 'track_108.wav', 'track_109.wav', 'track_11.wav', 'track_110.wav', 'track_111.wav', 'track_112.wav', 'track_113.wav', 'track_114.wav', 'track_115.wav', 'track_116.wav', 'track_117.wav', 'track_118.wav', 'track_119.wav', 'track_12.wav', 'track_13.wav', 'track_15.wav', 'track_17.wav', 'track_18.wav', 'track_19.wav', 'track_2.wav', 'track_21.wav', 'track_22.wav', 'track_23.wav', 'track_24.wav', 'track_25.wav', 'track_26.wav', 'track_29.wav', 'track_3.wav', 'track_31.wav', 'track_33.wav', 'track_34.wav', 'track_35.wav', 'track_37.wav', 'track_38.wav', 'track_39.wav', 'track_4.wav', 'track_40.wav', 'track_41.wav', 'track_42.wav', 'track_43.wav', 'track_44.wav', 'track_45.wav', 'track_46.wav', 'track_47.wav', 'track_48.wav', 'track_49.wav', 'track_5.wav', 'track_50.wav', 'track_52.wav', 'track_53.wav', 'track_54.wav', 'track_55.wav', 'track_56.wav', 'track_57.

In [18]:
instruments = ["cel", "pia", "tru", "vio", "voi"]

instruments_validation = [ "flu", "org", "sax", "gel", "gac", "cla"]

bound_instruments = {}
bound_instruments["cel"] = 0.9
bound_instruments["cla"] = 0.7
bound_instruments["gac"] = 0.35
bound_instruments["pia"] = 0.3
bound_instruments["tru"] = 0.6
bound_instruments["vio"] = 0.5
bound_instruments["voi"] = 0.25
bound_instruments["flu"] = 0.7
bound_instruments["org"] = 0.8
bound_instruments["gel"] = 0.5
bound_instruments["sax"] = 0.5

n_coef = 20

hop_length = 1024

dict_of_dict = {}

for file in file_names:

    file_path = os.path.join(directory, file)
    
    print(file_path)
    
    duration_seconds = librosa.get_duration(filename=file_path)
    print(duration_seconds)
    
    n_intervals = math.ceil(duration_seconds / 3)
    
    pred_labels_temp = np.zeros(n_intervals)

    dict_temp = {}

    for instrument_of_interest in instruments:
        
        model_name = instrument_of_interest +'best_model.h5' 

        model_temp = keras.models.load_model(model_name)

        for j in range(n_intervals-1):

            y, sr = librosa.load(file_path, mono=True, sr=None, offset=j*3.0, duration=3.0)

            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_coef, hop_length=hop_length)

            mfcc = mfcc.reshape(-1,n_coef,130,1)

            pred_labels_temp[j] = model_temp.predict(mfcc,)


        y, sr = librosa.load(file_path, mono=True, sr=None, offset=duration_seconds-3.0, duration=3.0)

        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_coef, hop_length=hop_length)

        mfcc = mfcc.reshape(-1,n_coef,130,1)

        pred_labels_temp[n_intervals-1] = model_temp.predict(mfcc,)

        print(pred_labels_temp)

        boundary = bound_instruments[instrument_of_interest]

        ones = np.count_nonzero(pred_labels_temp >= boundary)
        
        zeros = n_intervals - ones


        if (ones >= zeros):

            dict_temp[instrument_of_interest] = 1

        else:

            dict_temp[instrument_of_interest] = 0
            
    for instrument_of_interest in instruments_validation:

        model_name = instrument_of_interest +'_best_model_vali_test_AUGMENTATED_FINAL_test2.h5' 

        model_temp = keras.models.load_model(model_name)

        for j in range(n_intervals-1):

            y, sr = librosa.load(file_path, mono=True, sr=None, offset=j*3.0, duration=3.0)

            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_coef, hop_length=hop_length)

            mfcc = mfcc.reshape(-1,n_coef,130,1)

            pred_labels_temp[j] = model_temp.predict(mfcc,)


        y, sr = librosa.load(file_path, mono=True, sr=None, offset=duration_seconds-3.0, duration=3.0)

        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_coef, hop_length=hop_length)

        mfcc = mfcc.reshape(-1,n_coef,130,1)

        pred_labels_temp[n_intervals-1] = model_temp.predict(mfcc,)

        print(pred_labels_temp)
        
        boundary = bound_instruments[instrument_of_interest]

        ones = np.count_nonzero(pred_labels_temp >= boundary)
        
        zeros = n_intervals - ones

        if (ones >= zeros):

            dict_temp[instrument_of_interest] = 1

        else:

            dict_temp[instrument_of_interest] = 0
            

    dict_of_dict[file] = dict_temp

    
print(dict_of_dict)


C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_10.wav
7.0
1/1 [==============================] - 0s 22ms/step
[0.13766681 0.1032612  0.08211196]
1/1 [==============================] - 0s 9ms/step
[0.58555532 0.47748518 0.27767646]
1/1 [==============================] - 0s 10ms/step
[0.06703898 0.03596996 0.03418061]
1/1 [==============================] - 0s 13ms/step
[0.29144379 0.22562544 0.0660065 ]
1/1 [==============================] - 0s 11ms/step
[8.10037542e-04 5.15556196e-03 9.98876870e-01]
1/1 [==============================] - 0s 35ms/step
[0.01168249 0.33406943 0.2184523 ]
1/1 [==============================] - 0s 24ms/step
[0.01310721 0.00155036 0.0025073 ]
1/1 [==============================] - 0s 24ms/step
[1.40130442e-06 4.81726020e-05 5.12260249e-06]
1/1 [==============================] - 0s 18ms/step
[0.43164879 0.99711525 0.8992151 ]
1/1 [==============================] - 0s 22ms/step
[0.9966433  0.59778649 0.96209031]
1/1 [=======================

[0.74288481 0.99601144]
1/1 [==============================] - 0s 24ms/step
[7.39472825e-03 1.47180856e-07]
1/1 [==============================] - 0s 27ms/step
[0.0150958  0.60053587]
1/1 [==============================] - 0s 16ms/step
[0.4577336  0.15066318]
1/1 [==============================] - 0s 12ms/step
[2.45464027e-01 5.85366843e-06]
1/1 [==============================] - 0s 14ms/step
[0.00021909 0.00023442]
1/1 [==============================] - 0s 26ms/step
[1.74597005e-07 3.59613184e-09]
C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_109.wav
7.0
1/1 [==============================] - 0s 33ms/step
[0.12917058 0.11623535 0.10708227]
1/1 [==============================] - 0s 31ms/step
[0.13717806 0.09907977 0.08873689]
1/1 [==============================] - 0s 18ms/step
[0.02019036 0.17736593 0.15364966]
1/1 [==============================] - 0s 11ms/step
[0.13375495 0.14248282 0.15878534]
1/1 [==============================] - 0s 31ms/step
[0.13982958 0.94

1/1 [==============================] - 0s 21ms/step
[4.37781455e-05 5.98965585e-07 4.73686379e-09]
1/1 [==============================] - 0s 36ms/step
[0.50243253 0.64482689 0.00682992]
1/1 [==============================] - 0s 17ms/step
[0.89953601 0.0077828  0.04369725]
1/1 [==============================] - 0s 22ms/step
[6.85419013e-11 1.18403357e-10 2.24941680e-11]
C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_112.wav
10.0
1/1 [==============================] - 0s 29ms/step
[0.09996619 0.1101452  0.10613661 0.08966273]
1/1 [==============================] - 0s 17ms/step
[0.30226788 0.23756063 0.56795037 0.58200085]
1/1 [==============================] - 0s 19ms/step
[0.02185673 0.00930941 0.11913601 0.05316014]
1/1 [==============================] - 0s 13ms/step
[0.12350415 0.15376839 0.11976618 0.10651628]
1/1 [==============================] - 0s 25ms/step
[3.86308506e-02 1.73192546e-01 9.67330277e-01 9.58926103e-04]
1/1 [==============================] - 0s

1/1 [==============================] - 0s 23ms/step
[7.47976542e-11 8.59056836e-12]
C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_119.wav
8.0
1/1 [==============================] - 0s 33ms/step
[0.16277841 0.28501216 0.28504971]
1/1 [==============================] - 0s 28ms/step
[0.25900805 0.33250093 0.37371495]
1/1 [==============================] - 0s 16ms/step
[0.17761245 0.25063217 0.75068408]
1/1 [==============================] - 0s 32ms/step
[0.34829843 0.25088751 0.43386665]
1/1 [==============================] - 0s 16ms/step
[0.93053085 0.39594176 0.012065  ]
1/1 [==============================] - 0s 16ms/step
[4.69906269e-07 1.61926859e-04 3.56370956e-03]
1/1 [==============================] - 0s 36ms/step
[0.66635209 0.00892915 0.07203019]
1/1 [==============================] - 0s 24ms/step
[0.57399452 0.07474566 0.00068409]
1/1 [==============================] - 0s 19ms/step
[0.30402029 0.00331667 0.9999606 ]
1/1 [==============================] - 0s

1/1 [==============================] - 0s 31ms/step
[1.16704927e-04 7.51517321e-11 2.79153989e-09]
1/1 [==============================] - 0s 16ms/step
[0.9978444  0.19593173 0.07699127]
1/1 [==============================] - 0s 16ms/step
[7.58910730e-08 7.89115147e-05 3.91848971e-07]
1/1 [==============================] - 0s 31ms/step
[8.94495097e-05 2.47120298e-03 4.05156839e-04]
C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_17.wav
10.0
1/1 [==============================] - 0s 19ms/step
[0.18461627 0.14300552 0.2226585  0.18857646]
1/1 [==============================] - 0s 22ms/step
[0.26603085 0.17788415 0.2506026  0.2535935 ]
1/1 [==============================] - 0s 16ms/step
[0.10818715 0.08077427 0.62225389 0.05029443]
1/1 [==============================] - 0s 22ms/step
[0.77802932 0.54754215 0.29723704 0.35847846]
1/1 [==============================] - 0s 19ms/step
[0.08080782 0.4929423  0.00202825 0.16215311]
1/1 [==============================] - 0s 16ms

...

1/1 [==============================] - 0s 31ms/step
[1.57215325e-07 2.73381511e-06]
C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_67.wav
7.0
1/1 [==============================] - 0s 31ms/step
[0.21177199 0.21925478 0.19828051]
1/1 [==============================] - 0s 31ms/step
[0.22200117 0.25366014 0.15383503]
1/1 [==============================] - 0s 45ms/step
[0.49993026 0.23478886 0.59942466]
1/1 [==============================] - 0s 31ms/step
[0.26272637 0.33201933 0.26745611]
1/1 [==============================] - 0s 31ms/step
[0.00909678 0.00087348 0.00024364]
1/1 [==============================] - 0s 31ms/step
[0.00073299 0.00243298 0.0005391 ]
1/1 [==============================] - 0s 31ms/step
[0.00053324 0.00147973 0.14531489]
1/1 [==============================] - 0s 31ms/step
[7.41202603e-05 3.80813872e-04 1.79666331e-05]
1/1 [==============================] - 0s 31ms/step
[0.51583463 0.7407639  0.94314897]
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 27ms/step
[0.26397967 0.27764294 0.26271316]
1/1 [==============================] - 0s 16ms/step
[0.0313498  0.02937042 0.00556987]
1/1 [==============================] - 0s 28ms/step
[0.02190668 0.13001625 0.11832975]
1/1 [==============================] - 0s 35ms/step
[0.19838268 0.15484579 0.03571439]
1/1 [==============================] - 0s 26ms/step
[5.33959210e-05 2.61658244e-02 1.38309216e-02]
1/1 [==============================] - 0s 28ms/step
[0.05146229 0.05585781 0.11453502]
1/1 [==============================] - 0s 32ms/step
[1.67600644e-09 1.91192480e-13 2.29332826e-18]
1/1 [==============================] - 0s 28ms/step
[1.15521743e-06 9.25457400e-09 1.20333225e-05]
C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_70.wav
7.0
1/1 [==============================] - 0s 31ms/step
[0.13922329 0.12306841 0.07382922]
1/1 [==============================] - 0s 18ms/step
[0.11045518 0.39077336 0.31878424]
1/1 [==========

1/1 [==============================] - 0s 18ms/step
[0.04552638 0.65970278]
1/1 [==============================] - 0s 31ms/step
[1.18400276e-01 2.77000399e-05]
1/1 [==============================] - 0s 30ms/step
[0.09842999 0.97353214]
1/1 [==============================] - 0s 31ms/step
[0.81642848 0.01053371]
1/1 [==============================] - 0s 31ms/step
[1.17000903e-03 9.44515068e-06]
1/1 [==============================] - 0s 34ms/step
[8.56001563e-08 9.25760031e-01]
1/1 [==============================] - 0s 36ms/step
[3.16312730e-06 1.62568847e-06]
C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_79.wav
6.0
1/1 [==============================] - 0s 28ms/step
[0.14501311 0.19120333]
1/1 [==============================] - 0s 31ms/step
[0.08891284 0.13202935]
1/1 [==============================] - 0s 31ms/step
[0.03138453 0.08719181]
1/1 [==============================] - 0s 46ms/step
[0.44945085 0.3707552 ]
1/1 [==============================] - 0s 33ms/step
[

1/1 [==============================] - 0s 31ms/step
[9.95254510e-08 3.38432073e-05 3.59959920e-08]
C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_82.wav
6.0
1/1 [==============================] - 0s 34ms/step
[0.20020904 0.10530968]
1/1 [==============================] - 0s 33ms/step
[0.05570829 0.05743811]
1/1 [==============================] - 0s 31ms/step
[0.99273759 0.98423183]
1/1 [==============================] - 0s 31ms/step
[0.21550544 0.14658912]
1/1 [==============================] - 0s 31ms/step
[0.05971214 0.00352102]
1/1 [==============================] - 0s 31ms/step
[3.82644212e-05 7.10279835e-07]
1/1 [==============================] - 0s 31ms/step
[1.59656312e-02 9.60317848e-05]
1/1 [==============================] - 0s 31ms/step
[9.68420318e-06 5.37114397e-09]
1/1 [==============================] - 0s 31ms/step
[0.19057253 0.86911142]
1/1 [==============================] - 0s 16ms/step
[9.99919176e-01 3.44633087e-08]
1/1 [=========================

1/1 [==============================] - 0s 39ms/step
[0.22788702 0.24978875 0.16113622]
1/1 [==============================] - 0s 22ms/step
[0.92436248 0.99074793 0.7743637 ]
1/1 [==============================] - 0s 30ms/step
[0.27675399 0.11472674 0.40109977]
1/1 [==============================] - 0s 31ms/step
[0.00122595 0.00104942 0.00128399]
1/1 [==============================] - 0s 31ms/step
[4.78308721e-06 2.37635279e-04 4.19407530e-04]
1/1 [==============================] - 0s 31ms/step
[0.01157686 0.93807101 0.00734302]
1/1 [==============================] - 0s 22ms/step
[1.17818388e-04 1.42839644e-05 1.80865541e-07]
1/1 [==============================] - 0s 17ms/step
[0.14127776 0.33864146 0.46186706]
1/1 [==============================] - 0s 32ms/step
[3.06992604e-07 1.69601506e-07 1.15474442e-07]
1/1 [==============================] - 0s 31ms/step
[3.80046605e-10 2.47043005e-07 3.71063652e-05]
C:/Users/Lovric/Downloads/drive-download-20230507T145001Z-001/track_90.wav
6.0
1/1

In [21]:
file_path = "LUMENDS2023_Final_predictions.json"

# Open the file in write mode and write the JSON data
with open(file_path, "w") as json_file:
    json.dump(dict_of_dict, json_file, indent=4)